# Making the train and test datasets and running statistics on them
- You can uncomment a whole cell by pressing ctrl+A (while inside the cell) and then pressing ctrl+/
- The cells are greyed out to not accidentally run them, since we only wanted to run each cell once.

In [2]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
#from google.colab import drive
#drive.mount('/content/gdrive')
import time
from scipy import io
import scipy.stats as stats  

# Making the train and test datasets 

In [ ]:
# ##ONLY RUN ONCE

# loaded = io.loadmat('clouds/2D_zeroPadaed_clouds_orig.mat')
# total_clouds = [value for key, value in loaded.items() if key.startswith("cloud_")]
# clouds = [cloud for cloud in total_clouds if (np.count_nonzero(cloud) > 200)] #remove clouds that are nearly empty
# print(total_clouds[0].shape)
# print("Number of total clouds: " + str(len(total_clouds))) 
# print("Number of non-empty clouds: " + str(len(clouds)))
# tensor_clouds = torch.Tensor(clouds) # transform to torch tensor
# tensor_clouds = tensor_clouds.unsqueeze(1)

# # Create the dataset
# dataset = torch.utils.data.TensorDataset(tensor_clouds)

# train_size = int(0.9 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
# torch.save(train_dataset, os.path.join(os.getcwd() + '/dataset' , 'train.pt'))
# torch.save(test_dataset, os.path.join(os.getcwd() + '/dataset' , 'test.pt'))

# Betas mean
- Calculate the mean of the betas in each image.
- Find the gamma parameters that fit this distribution.
- Plot the histogram.

In [3]:
# train_dataset = torch.load(os.path.join(os.getcwd() + '/dataset' , 'train.pt'))
# train_size = len(train_dataset)
# betas = []
# for i in range(train_size):
#     for j in range(130):
#         for k in range(80):
#             if train_dataset[i][0][0][j][k] != 0:
#                betas.append(train_dataset[i][0][0][j][k])
# np_betas = [x.numpy() for x in betas]
# np_betas = np.array(np_betas, dtype=np.float32)
# io.savemat('betas.mat', {'np_betas': np_betas})
# fit_alpha, fit_loc, fit_beta=stats.gamma.fit(np_betas)
# gamma_para = [fit_alpha, fit_loc, fit_beta]
# io.savemat('gamma.mat', {'gamma_para': gamma_para})
# np_betas = io.loadmat('betas.mat')['np_betas']
# hist, bin_edges = np.histogram(np_betas, density=True)
# plt.plot(hist) # plotting by columns
# plt.show()

In [4]:
# test_dataset = torch.load(os.path.join(os.getcwd() + '/dataset' , 'test.pt'))
# test_size = len(test_dataset)
# betas = []
# for i in range(test_size):
#     for j in range(130):
#         for k in range(80):
#             if test_dataset[i][0][0][j][k] != 0:
#                betas.append(test_dataset[i][0][0][j][k])
# np_betas = [x.numpy() for x in betas]
# np_betas = np.array(np_betas, dtype=np.float32)
# io.savemat('betas_test.mat', {'np_betas': np_betas})
# fit_alpha, fit_loc, fit_beta=stats.gamma.fit(np_betas)
# gamma_para = [fit_alpha, fit_loc, fit_beta]
# io.savemat('gamma_test.mat', {'gamma_para': gamma_para})
# np_betas = io.loadmat('betas_test.mat')['np_betas']
# hist, bin_edges = np.histogram(np_betas, density=True)
# plt.plot(hist) # plotting by columns
# plt.show()

In [4]:
# train_alpha, train_loc, train_beta  = io.loadmat('gamma.mat')['gamma_para'][0]
# train_prob = torch.distributions.gamma.Gamma(train_alpha, train_beta, validate_args=None)
# #
# test_alpha, test_loc, test_beta  = io.loadmat('gamma_test.mat')['gamma_para'][0]
# test_prob = torch.distributions.gamma.Gamma(test_alpha, test_beta, validate_args=None)

# torch.distributions.kl.kl_divergence(test_prob, train_prob)

tensor(0.0004)

In [5]:
# torch.distributions.kl.kl_divergence(train_prob,test_prob)

tensor(0.0004)

### Calculate some beta stats for the test dataset.

In [9]:
# test_dataset = torch.load(os.path.join(os.getcwd() + '/dataset' , 'test.pt'))
# test_size = len(test_dataset)
# sa = []
# for i in test_dataset:
#     sa.append(i[0][0])
# data_Tensor = torch.stack(sa,0)

# var = torch.var(data_Tensor)
# mean = torch.mean(data_Tensor)
# data_min = torch.min(data_Tensor)
# data_max = torch.max(data_Tensor)

In [11]:
# test_dataset = torch.load(os.path.join(os.getcwd() + '/dataset' , 'test.pt'))
# test_size = len(test_dataset)
# mins = []
# for i in range(test_size):
#     tmp = 100000
#     for j in range(130):
#         for k in range(80):
#             if test_dataset[i][0][0][j][k] != 0 and test_dataset[i][0][0][j][k] < tmp:
#                 tmp =  test_dataset[i][0][0][j][k]
#     mins.append(tmp)
# mins = [x.numpy() for x in mins]
# mins = np.array(mins, dtype=np.float32)

## Non-zero elements in the training set.
- Count the number of non-zero elements in every image.
- Subtract 201 and find the gamma parameters that fit this distribution.
- Plot the histogram and save the gamma parameters.

In [4]:
# train_dataset = torch.load(os.path.join(os.getcwd() + '/dataset' , 'train.pt'))
# train_size = len(train_dataset)

# non_zero_count_real = []
# for i in range(train_size):
#     non_zero_count_real.append(torch.count_nonzero(train_dataset[i][0]))
    
# non_zero_count_real = torch.stack(non_zero_count_real).numpy()
# #substract 201 so that it starts from 0
# non_zero_count_real = np.subtract(non_zero_count_real, 201.0)

# plt.hist(non_zero_count_real, bins = 120)
# plt.title('Training images')
# plt.xlabel(r'$\mathrm{(Shifted\ left)\ Non-zero\ count\ of\:}\ \beta$')


# fit_alpha, fit_loc, fit_beta=stats.gamma.fit(non_zero_count_real)
# non_zero_train_gamma = [fit_alpha, fit_loc, fit_beta]
# io.savemat('non_zero_train_gamma.mat', {'non_zero_train_gamma': non_zero_train_gamma})

### Do the same for the test set.

In [15]:
test_dataset = torch.load(os.path.join(os.getcwd() + '/dataset' , 'test.pt'))
test_size = len(test_dataset)

non_zero_count_real_test = []
for i in range(test_size):
    non_zero_count_real_test.append(torch.count_nonzero(test_dataset[i][0]))
    

non_zero_count_real_test = torch.stack(non_zero_count_real_test).numpy()


fit_alpha, fit_loc, fit_beta=stats.gamma.fit(non_zero_count_real_test)
non_zero_test_gamma = [fit_alpha, fit_loc, fit_beta]
io.savemat('non_zero_test_gamma.mat', {'non_zero_test_gamma': non_zero_test_gamma})

In [16]:
train_alpha, train_loc, train_beta  = io.loadmat('non_zero_train_gamma.mat')['non_zero_train_gamma'][0]
train_prob = torch.distributions.gamma.Gamma(train_alpha, train_beta, validate_args=None)
#
test_alpha, test_loc, test_beta  = io.loadmat('non_zero_test_gamma.mat')['non_zero_test_gamma'][0]
test_prob = torch.distributions.gamma.Gamma(test_alpha, test_beta, validate_args=None)

torch.distributions.kl.kl_divergence(test_prob, train_prob)

tensor(0.0154)

### Calculate some beta stats for the training dataset.

In [7]:
train_dataset = torch.load(os.path.join(os.getcwd() + '/dataset' , 'train.pt'))
train_size = len(train_dataset)
sa = []
for i in train_dataset:
    sa.append(i[0][0])
data_Tensor = torch.stack(sa,0)

var = torch.var(data_Tensor)
mean = torch.mean(data_Tensor)
data_min = torch.min(data_Tensor)
data_max = torch.max(data_Tensor)